In [2]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [18]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
#     add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
#                          'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
#                         }
    add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['submission in progress', 'current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            

dekker_planned = [
'694d56b6-a590-47a5-a1c4-7fe879a23ed9',
'521b16b4-6bfc-4098-8d7d-08062384cef2',
'4a2c06d5-0b55-4dec-b117-45fb97f91173',
'5797093d-7d8d-4f25-a42d-b49f8114c4cc',
'1311c73f-82e4-4b21-8ded-358c4a301195',
]

my_exp = dekker_planned[4]

print my_exp
store_dict = {} 
item_uuids = []
store, uuids = record_object(my_exp, store_dict, item_uuids)
print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




1311c73f-82e4-4b21-8ded-358c4a301195
experiment_set_replicate 1311c73f-82e4-4b21-8ded-358c4a301195 planned
experiment_hi_c 49aa3b78-2e3e-47dd-ac42-ba6a723e4ea8 planned
sop_map 39373842-12e9-4bbb-8f70-270ec1a859ea deleted
biosample 30dc8f07-91ae-491e-add5-c7011b2590be deleted
biosample_cell_culture ef9b83e9-0e3c-40e9-90ea-714308d7495a deleted
32
32


In [19]:
## This part is only for making non file objects "in submission"
counter = 0
for a_type in store:

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print '    DELETED DATA', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "deleted"}
            #ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

experiment_set_replicate planned 1311c73f-82e4-4b21-8ded-358c4a301195
    DELETED DATA biosample_cell_culture ef9b83e9-0e3c-40e9-90ea-714308d7495a
experiment_hi_c planned 49aa3b78-2e3e-47dd-ac42-ba6a723e4ea8
    DELETED DATA sop_map 39373842-12e9-4bbb-8f70-270ec1a859ea
    DELETED DATA biosample 30dc8f07-91ae-491e-add5-c7011b2590be
2


In [20]:
## This part is only for making non file objects "in submission"
counter = 0
for a_type in store:

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print '    DELETED DATA', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "deleted"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

experiment_set_replicate planned 1311c73f-82e4-4b21-8ded-358c4a301195
    DELETED DATA biosample_cell_culture ef9b83e9-0e3c-40e9-90ea-714308d7495a
experiment_hi_c planned 49aa3b78-2e3e-47dd-ac42-ba6a723e4ea8
    DELETED DATA sop_map 39373842-12e9-4bbb-8f70-270ec1a859ea
    DELETED DATA biosample 30dc8f07-91ae-491e-add5-c7011b2590be
2
